In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_distances
from itertools import combinations
from collections import defaultdict
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD


# 데이터 로딩
orders = pd.read_csv("./data/Sample_InputData.csv")
od_matrix = pd.read_csv("./data/Sample_OD_Matrix.csv", index_col=0)
parameters = pd.read_csv("./data/Sample_Parameters.csv")

# 1. 연관성 분석 & 빈도 기반 클러스터링 배치

# 파라미터 설정
rk = int(parameters.loc[parameters["PARAMETERS"] == "RK", "VALUE"].values[0])
start_node = od_matrix.index[0]
rack_columns = [col for col in od_matrix.columns if col.startswith("WP_")]

# 1. SKU 간 동시 출현 행렬
orders_by_ordno = orders.groupby("ORD_NO")["SKU_CD"].apply(list)
sku_list = sorted(orders["SKU_CD"].unique())
sku_index = {sku: idx for idx, sku in enumerate(sku_list)}
co_occurrence = np.zeros((len(sku_list), len(sku_list)), dtype=int)

for sku_group in orders_by_ordno:
    for sku1, sku2 in combinations(sku_group, 2):
        idx1, idx2 = sku_index[sku1], sku_index[sku2]
        co_occurrence[idx1][idx2] += 1
        co_occurrence[idx2][idx1] += 1
    for sku in sku_group:
        idx = sku_index[sku]
        co_occurrence[idx][idx] += 1

co_matrix = pd.DataFrame(co_occurrence, index=sku_list, columns=sku_list)

# 2. 클러스터링 (Agglomerative, cosine 거리)
cosine_dist = cosine_distances(co_matrix)
clustering = AgglomerativeClustering(n_clusters= None, metric='precomputed', linkage='average', distance_threshold= 0.3) 
# 작을수록 더 많은 클러스터 : 0.3일때 330개의 군집, 0.2일때 334, 0.1일때 336, 0.05일떄 336, 0.4일 때 311개, 0.5일 때 255개
cluster_labels = clustering.fit_predict(cosine_dist)
sku_cluster_map = pd.DataFrame({"SKU": co_matrix.index, "Cluster": cluster_labels})

print("총 생성된 클러스터 수:", sku_cluster_map['Cluster'].nunique())

# 3. 클러스터 대표 빈도 = 클러스터 내 SKU 중 최대 출현 빈도
sku_freq = orders["SKU_CD"].value_counts().to_dict()
cluster_freq = sku_cluster_map.copy()
cluster_freq["Frequency"] = cluster_freq["SKU"].map(sku_freq)
cluster_max_freq = cluster_freq.groupby("Cluster")["Frequency"].max().sort_values(ascending=False)
cluster_max_freq # 253 클러스터가 빈도수 10, 223 클러스터가 빈도수 1

# 4. 상위 20% 클러스터 선택
# top_n = int(len(cluster_max_freq) * 0.8) or 1
# top_clusters = cluster_max_freq.head(top_n).index.tolist()



# 5. 우선 배치 대상 SKU
priority_skus = sku_cluster_map[sku_cluster_map["Cluster"].isin(top_clusters)]["SKU"].tolist()

# n_full_racks = len(priority_skus) // rk
# priority_skus_to_assign = priority_skus[:n_full_racks * rk]  # 완전히 채울 수 있는 SKU만 우선 배치
# remaining_priority_skus = priority_skus[n_full_racks * rk:]  # 나머지는 MIT로 넘기기

# 6. 입출고지점 인접 랙에 우선 배치
rack_distances = od_matrix.loc[start_node, rack_columns].sort_values()
rack_iter = iter(rack_distances.index)
rack_sku_allocation = defaultdict(list)
current_rack = next(rack_iter)

for sku in priority_skus:
    while len(rack_sku_allocation[current_rack]) >= rk: # 1랙당 rk만큼 배치 가능
        current_rack = next(rack_iter)
    rack_sku_allocation[current_rack].append(sku)

# 7. 결과 정리
priority_alloc_df = pd.DataFrame(
    [(sku, rack) for rack, skus in rack_sku_allocation.items() for sku in skus],
    columns=["SKU", "Assigned_Rack"]
)

# 결과 확인
print(priority_alloc_df.head())

# 2. 남은 SKU / 남은 랙에 MIT로 배치
sku_list = set(sku_list)
used_skus  = set(priority_alloc_df['SKU'])
mit_skus = sorted(sku_list - used_skus)
len(mit_skus) # 270개가 남음 - 상위20% 클러스터 / 170개가 남음 - 상위50% 클러스터

used_racks = set(priority_alloc_df['Assigned_Rack'])
mit_racks = sorted(set(rack_columns) - used_racks)
len(mit_racks) # 135개

orders["LOC"] = orders["SKU_CD"].map(dict(zip(priority_alloc_df["SKU"], priority_alloc_df["Assigned_Rack"])))
orders.to_csv("./MIT_Hybrid_Output.csv", index=False)